In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000


from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [4]:
#
columns = ['acoustic_data','time_to_failure']



In [5]:
train_data, val_data = train_test_split(splits, test_size=0.1, random_state=42)

In [6]:
TIMESTEPS=150000
BATCH_SIZE=16
NUMBER_OF_BATCHES = int(np.ceil(len(train_data)/BATCH_SIZE))
NUMBER_OF_VALIDATION_STEPS = int(np.ceil(len(val_data) / BATCH_SIZE))

In [7]:
train_data_batch = np.array_split(train_data, NUMBER_OF_BATCHES)
val_data_batch = np.array_split(val_data, NUMBER_OF_VALIDATION_STEPS)

In [8]:
NOISE=0.75

def add_noise(dff, pct=NOISE):
    mu = dff['acoustic_data'].mean()
    sigma = dff['acoustic_data'].std()

    indices = np.random.choice(dff.index.values, int(len(dff)*pct))
    dff.loc[indices, 'acoustic_data'] = np.random.normal(mu, sigma, len(indices)) 
    return dff


def get_batch(list_of_files):
#     batch = np.empty((len(list_of_files),TIMESTEPS,1),dtype=float)
#     target = np.empty((len(list_of_files),1),dtype=float)
    #print(list_of_files)
    batch = []
    target = []

    for idx, file in enumerate(list_of_files):
        #print(idx,file)
        path = f'train/{file}'
        df = pd.read_csv(path, float_precision='round_trip', header=None)
        df.columns = columns
        df[['acoustic_data']] = StandardScaler().fit_transform(df[['acoustic_data']].astype('float'))
        #print(df.head())
        #print(len(batch))
        batch.append(df['acoustic_data'].values)
        target.append(df['time_to_failure'].values[-1])
        df_noise = add_noise(df)
        #print(df_noise.head())
        batch.append(df_noise['acoustic_data'].values)#.reshape(-1,TIMESTEPS,1))
        target.append(df_noise['time_to_failure'].values[-1])#.reshape(-1,1))
        #print(np.array(batch).reshape(-1,TIMESTEPS,1).shape)
        #batch = np.array(batch).reshape(-1,TIMESTEPS,1)
        #target = np.array(target).reshape(-1,1)
    return (batch, target)

In [9]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, list_of_files, steps,name):
        self.list_of_files = list_of_files
        self.steps = steps
        self.name = name

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return self.steps

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
#         if self.name == 'val':
        #print('idx', idx)
        #print("DEBUG", self.list_of_files[idx])
        #if idx == len(self.list_of_files):
            #print(idx, self.list_of_files)
        train,Y = get_batch(self.list_of_files[idx])
        #print(np.array(train).reshape(-1,TIMESTEPS,1).shape)
        #print("idx",idx)
        #print("LOLILOL")
        #print(train.shape, Y.shape)
        train = np.array(train).reshape(-1,TIMESTEPS,1)
        #print(train.shape)
        Y = np.array(Y).reshape(-1,1)

        return (train,Y)

Using TensorFlow backend.


In [ ]:
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,Conv1D,TimeDistributed,MaxPooling1D,Flatten,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional

from keras import backend as K
K.clear_session()

TIMESTEPS=150000

dropout=0.2

kernel_size=5
filters=10
strides=5
pool_size=2

my_model = Sequential()
my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, #activation='relu',
               strides=strides, input_shape=(TIMESTEPS,1))
)
             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())

my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, #activation='relu',
               strides=strides, input_shape=(TIMESTEPS,1))
)             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())

my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, #activation='relu',
               strides=strides, input_shape=(TIMESTEPS,1))
)             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())


my_model.add(GRU(units = 8,dropout=dropout,recurrent_dropout=dropout))

my_model.add(Dense(1))



my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, verbose=0),
    #Reseter()
]

my_training_batch_generator = MY_Generator(train_data_batch, NUMBER_OF_BATCHES, 'train')
my_validation_batch_generator = MY_Generator(val_data_batch, NUMBER_OF_VALIDATION_STEPS, 'val')


history = my_model.fit_generator(generator=my_training_batch_generator,
                                          #steps_per_epoch=NUMBER_OF_BATCHES,
                                          epochs=500,
                                          validation_data=my_validation_batch_generator,
                                          #validation_steps=NUMBER_OF_VALIDATION_STEPS,
                                          callbacks=callbacks,
                                          shuffle=True,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          use_multiprocessing=True,
                                          workers=8,
                                          #max_queue_size=32
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 30000, 10)         60        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15000, 10)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15000, 10)         40        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3000, 10)          510       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1500, 10)          0         
_________________________________________________________________
batch_normalization_2

Epoch 33/500
236/236 [==============================] - 63s 268ms/step - loss: 2.4137 - mean_absolute_error: 2.4137 - val_loss: 2.6802 - val_mean_absolute_error: 2.6802
Epoch 34/500
236/236 [==============================] - 63s 268ms/step - loss: 2.3908 - mean_absolute_error: 2.3908 - val_loss: 2.7315 - val_mean_absolute_error: 2.7315
Epoch 35/500
236/236 [==============================] - 64s 270ms/step - loss: 2.3929 - mean_absolute_error: 2.3929 - val_loss: 3.7560 - val_mean_absolute_error: 3.7560
Epoch 36/500
236/236 [==============================] - 64s 270ms/step - loss: 2.3683 - mean_absolute_error: 2.3683 - val_loss: 2.5474 - val_mean_absolute_error: 2.5474
Epoch 37/500
236/236 [==============================] - 63s 269ms/step - loss: 2.3634 - mean_absolute_error: 2.3634 - val_loss: 2.5316 - val_mean_absolute_error: 2.5316
Epoch 38/500
236/236 [==============================] - 64s 269ms/step - loss: 2.3649 - mean_absolute_error: 2.3649 - val_loss: 2.9477 - val_mean_absolute_

236/236 [==============================] - 64s 270ms/step - loss: 2.3006 - mean_absolute_error: 2.3006 - val_loss: 2.3120 - val_mean_absolute_error: 2.3120
Epoch 80/500
236/236 [==============================] - 64s 271ms/step - loss: 2.2760 - mean_absolute_error: 2.2760 - val_loss: 2.9535 - val_mean_absolute_error: 2.9535
Epoch 81/500
236/236 [==============================] - 63s 269ms/step - loss: 2.2695 - mean_absolute_error: 2.2695 - val_loss: 2.5588 - val_mean_absolute_error: 2.5588
Epoch 82/500
236/236 [==============================] - 64s 270ms/step - loss: 2.2881 - mean_absolute_error: 2.2881 - val_loss: 3.2823 - val_mean_absolute_error: 3.2823
Epoch 83/500
236/236 [==============================] - 64s 270ms/step - loss: 2.2736 - mean_absolute_error: 2.2736 - val_loss: 2.2653 - val_mean_absolute_error: 2.2653
Epoch 84/500
236/236 [==============================] - 64s 269ms/step - loss: 2.2744 - mean_absolute_error: 2.2744 - val_loss: 2.3519 - val_mean_absolute_error: 2.3519

In [ ]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [ ]:
test_splits

In [ ]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

In [ ]:
ids = []
preds = []
i=0
for r in res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

In [ ]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()